In [71]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# dependencies
import pymongo as pm

In [3]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

# define db and collections
DB_NAME = 'scrape'
db = client[DB_NAME]

In [4]:
# this collection will store all the HTML pages
COL_NAME = 'msnbc'
htmlCol = db[COL_NAME]

In [6]:
# import db into local mongo instance
PATH = 'db/json/by_media/left/'
source = 'msnbc.json'

!mongoimport --db {DB_NAME} --collection {COL_NAME} --file {PATH+source} --batchSize 1

2018-06-16T21:33:37.706-0400	connected to: localhost
2018-06-16T21:33:40.690-0400	[################........] scrape.msnbc	338MB/485MB (69.8%)
2018-06-16T21:33:41.910-0400	[########################] scrape.msnbc	485MB/485MB (100.0%)
2018-06-16T21:33:41.911-0400	imported 1586 documents


In [110]:
i = 0
total_docs = htmlCol.find().count()
for doc in htmlCol.find():
    #print(f"{doc['url']} : {doc['html'][:10]}")
    if 'dek___3AQpw' in doc['html']:
        i += 1
    else:
        print(doc['_id'])
        break
print(f'There are: {i} date features out of {total_docs} docs')

5b243fb2897e82028b8ffe50
There are: 0 date features out of 1586 docs


In [119]:
from bson.objectid import ObjectId

htmlCol.find({'_id':ObjectId('5b243fb2897e82028b8ffe50')})[0]

{'_id': ObjectId('5b243fb2897e82028b8ffe50'),
 'url': 'http://www.msnbc.com/',
 'html': '<!DOCTYPE html>\n<html xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml">\n  <head>\n    <!--[if IE]><![endif]-->\n<meta charset="utf-8" />\n<meta name="twitter:description" content="Breaking news and in-depth analysis of the headlines, as well as commentary and informed perspectives from The Rachel Maddow Show, Morning Joe &amp; more." />\n<meta name="twitter:account_id" content="2836421" />\n<meta name="twitter:domain" content="msnbc.com" />\n<meta name="twitter:creator:id" content="2836421" />\n<link rel="shortcut icon" href="http://www.msnbc.com/sites/msnbc/themes/leanforward/favicon.ico" />\n<meta name="twitter:site" content="@msnbc" />\n<meta name="twitter:site:id" content="2836421" />\n<meta name="twitter:card" content="summary_large_image" />\n<meta property="og:image" content="http://www.msnbc.com/sites/msnbc/themes/leanforward/images/site-header/

In [116]:
for i in htmlCol.find({'_id':'5b243fb2897e82028b8ffe50'}):
    print(i)

In [13]:
from bs4 import BeautifulSoup as bs

In [30]:
# pymongo 'find' returns cursor that allows iterating through results
# calling first object [0] allows accessing the dictionary with results
# the ['html'] is the key in the dictionary
html = htmlCol.find({'url':'http://www.msnbc.com/velshi-ruhle/watch/jeff-sessions-is-justifying-harsh-immigration-policy-with-the-bible-1256689731629'},\
            projection={'html':True, '_id':False})[0]['html']

In [33]:
sp = bs(html, 'lxml')

In [46]:
sp.find('p', class_='dek___3AQpw').text[:-len(sp.find('p', class_='dek___3AQpw').span.text)]

'Attorney General Jeff Sessions is using the Bible to justify separating families at the border. Father James Martin joins Ali Velshi to discuss what the Bible actually says about migrants and refugees.'

In [44]:
sp.find('p', class_='dek___3AQpw').span.text

'Jun.15.2018'

In [69]:
import newspaper

In [55]:
article = Article('')
article.set_html(html)
article.parse()

In [56]:
article.authors

[]

In [74]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/yegor3/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [75]:
article.build()

In [83]:
article.title

'Jeff Sessions is justifying harsh immigration policy with the Bible'

In [59]:
msnbc_paper = newspaper.build('http://www.msnbc.com/')

In [63]:
msnbc_paper.size()

142

In [65]:
for category in msnbc_paper.feed_urls():
    print(category)